In [1]:
import requests
import apimoex
import ts2vg as ts
from igraph import *
import numpy as np
import networkx as nx
import igraph as ig
import cairocffi
import matplotlib.pyplot as plt
import pandas as pd
import dill
import json
import datetime

In [5]:
def download_stock_market_data(url, file_path):
    with requests.get(url) as response:
        if response.status_code == 200:
            with open(file_path, 'w') as file:
                json.dump(response.json(), file)

In [6]:
url_ofz = 'https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQOB/securities.json?first=200'
url_bonds = 'https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQCB/securities.json?first=3000'
download_stock_market_data(url_ofz, './data/cmm_bonds')

In [8]:
ofz_path = './data/ofz_bonds'
cmm_path = './data/cmm_bonds'

In [9]:
with open(ofz_path, 'rb') as file:
    ofz = json.load(file)
with open(cmm_path, 'rb') as file:
    cmm = json.load(file)

In [25]:
cb_ir = pd.read_csv('data.csv', delimiter=';', header=None)
cb_ir.columns = ['TRADEDATE', 'IR']
cb_ir.set_index('TRADEDATE', inplace=True)
cb_ir.index = pd.to_datetime(cb_ir.index, dayfirst=True)

In [10]:
ofz['securities']['data'][33][0]

'SU29010RMFS4'

In [11]:
selected_cols = ['FACEVALUE', 'MATDATE', 'TRADEDATE', 'CLOSE', 'VOLUME']

In [28]:
i

48

In [29]:
bonds_dict = {}
with requests.Session() as session:
    for i, bond in enumerate(ofz['securities']['data']):
        if i < 48:
            continue
        print(bond[0])
        bonds_dict[bond[0]] = {}
        data = apimoex.get_board_history(session, security=ofz['securities']['data'][i][0], board='TQOB', market='bonds', columns=selected_cols)
        df = pd.DataFrame(data).dropna(axis=0)
        if len(df) == 0:
            continue
        bonds_dict[bond[0]]['facevalue'] = df['FACEVALUE'].iloc[0]
        bonds_dict[bond[0]]['maturity'] = df['MATDATE'].iloc[0]
        
            
        df.set_index('TRADEDATE', inplace=True)
        df.index = pd.to_datetime(df.index)
        
        mat_date = datetime.datetime.date( pd.to_datetime(bonds_dict[bond[0]]['maturity']) )
        df['COUPON'] = 0
        df.loc[mat_date, 'COUPON'] = 0
        
        print('initial offering date')
        start_date    = input()
        bonds_dict[ofz['securities']['data'][i][0]]['offering'] = start_date
        start_date = datetime.datetime.date(pd.to_datetime(start_date, dayfirst=True))
        
        print('arbitraty coupon payoff date')
        next_payment  = datetime.datetime.date(pd.to_datetime(input(), dayfirst=True))
        print('the next coupon payoff date')
        next_payment2 = datetime.datetime.date(pd.to_datetime(input(), dayfirst=True))
        coupon_period = next_payment2 - next_payment
        print('coupon value')
        coupon = float(input().replace(',','.'))
        print('payoff periodicity')
        periodicity = float(input())
        bonds_dict[bond[0]]['coupon'] = coupon
        bonds_dict[bond[0]]['periodicity'] = periodicity
        
        print(next_payment2 - next_payment)
        
        while start_date <= mat_date + datetime.timedelta(days=25):
            df.loc[pd.to_datetime(start_date), 'COUPON'] = coupon
            start_date += coupon_period
        df.loc[mat_date, 'COUPON'] += bonds_dict[bond[0]]['facevalue']
            
        df['PROFITABILITY'] = periodicity * coupon / df['CLOSE'] / 100
            
        df['IR'] = 0
        for date in cb_ir.index[::-1]:
            try:
                df.loc[date:, 'IR'] = cb_ir.loc[date][0] / 100
            except:
                continue  
                
        # df['DISCOUNT_FACTOR'] = 0
        # df.loc[df.index[-1], 'DISCOUNT_FACTOR'] = 1
        # for i, date in enumerate(df.index[:-2:-1]):
        #     df.loc[date, 'DISCOUNT_FACTOR'] = np.exp(
        #         df.loc[date, 'IR'] * (df.index[-1-i] - date).days / 365.25
        #     ) * df.loc[df.index[-1-i], 'DISCOUNT_FACTOR']
        
        df = df.drop(columns=['FACEVALUE', 'MATDATE'])
        bonds_dict[bond[0]]['market_data'] = df
        
        with open(f'./data/{bond[0]}', 'wb') as bfile:
            dill.dump(bonds_dict[bond[0]], bfile)

SU46012RMFS9
initial offering date


 14.02.2003


arbitraty coupon payoff date


 14.09.2022


the next coupon payoff date


...

coupon value


 12,95


payoff periodicity


 2


182 days, 0:00:00


In [ ]:
drop_set = ['SU29013RMFS8', 'SU29014RMFS6', 'SU29015RMFS3',
           'SU29016RMFS1', 'SU29017RMFS9', 'SU29018RMFS7',
           'SU29019RMFS5', 'SU29020RMFS3', 'SU29021RMFS1',
           'SU29022RMFS9', 'SU29024RMFS5', 'SU29024RMFS5',
           'SU29025RMFS2', 'SU46011RMFS1', 'SU46012RMFS9',
           'SU46023RMFS6', ]

In [451]:
with open('./data/SU26207RMFS9', 'rb') as file:
    cmm = dill.load(file)

In [ ]:
cmm['market_data']

,CLOSE,VOLUME,COUPON,PROFITABILITY,IR
TRADEDATE,,,,,
2013-03-25,108.00,20.0,0.00,0.752593,0.00
2013-04-04,108.10,1000.0,0.00,0.751896,0.00
2013-06-17,103.75,1.0,0.00,0.783422,0.00
2013-08-19,103.75,1.0,0.00,0.783422,0.00
2013-08-22,102.20,1.0,0.00,0.795303,0.00
...,...,...,...,...,...
2025-02-05,NaN,NaN,40.64,NaN,0.13
2025-08-06,NaN,NaN,40.64,NaN,0.13
2026-02-04,NaN,NaN,40.64,NaN,0.13


In [26]:
import yfinance as yf

In [36]:
forex_data = yf.download('USDRUB=X', start='2000-01-01', end=datetime.datetime.now())

[*********************100%***********************]  1 of 1 completed


In [39]:
forex_data = forex_data.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])

In [95]:
with open('./data/usdrub', 'wb') as file:
    dill.dump(fx, file)

In [77]:
with open('./data/usdrub', 'rb') as file:
    fx = dill.load(file)